# Electrical Circuit Battery Model

## Import Libraries

In [ ]:
%matplotlib inline

import sys
import matplotlib.pyplot as plt

sys.path.append("../")
from csvhelper import *
from measurement import *
from isimulator import *

## Example 1: Continuous Resistor Load
Discharging an Alkaline battery with a $10\Omega$ continuous resistor load.

In [ ]:
R = 10 #Ohm
t, E_Batt, U_Batt, I_Batt = ISimulator(Alkaline(), ResistorLoad(R)).run()
  
plt.figure()
plt.xlabel('t [h]')
plt.ylabel('$U_{Batt}$ [V]')
plt.plot([x / 3600.0 for x in t], U_Batt)
  
plt.figure()
plt.xlabel('t [h]')
plt.ylabel('$I_{Batt}$ [mA]')
plt.plot([x / 3600.0 for x in t], [x * 1000.0 for x in I_Batt])  
    
plt.figure()
plt.xlabel('$E_{Batt}$ [Wh]')
plt.ylabel('$U_{Batt}$ [V]')
plt.plot([x / 3600.0 for x in E_Batt], U_Batt)

## Example 2: Continuous Resistor Load (simple)
Discharging an Alkaline battery with a $10\Omega$ continuous resistor load.
Print end values only to increase simulation speed and reduce memory requirements.

In [ ]:
R = 10 #Ohm
t_cutoff, E_cutoff = ISimulator(Alkaline(), ResistorLoad(R)).run(simple=True)
  
print("t_cutoff = {} h".format(t_cutoff / 3600.0))
print("E_cutoff = {} Wh".format(E_cutoff / 3600.0))

## Example 3: Intermittent Constant Power Load
Discharging a Li-Ion battery with a 1 Watt constant power load ($T_{ON} = T_{OFF} = 1h$).

You can also change the discrete simulation interval `step` (default: 1s), the circuit model update interval `mstep` (default: `step`), and the interval the battery values are written to the result arrays `astep` (default: `step`). All values in seconds.

In [ ]:
P = 1.0 #Watt
T_ON = T_OFF = 3600.0 #s
t, E_Batt, U_Batt, I_Batt = ISimulator(LiPo(), ConstantPowerLoad(P, T_ON, T_OFF))\
    .run(step = 0.1, mstep = 1.0, astep = 1.0)

plt.figure()
plt.xlabel('t [h]')
plt.ylabel('$U_{Batt}$ [V]')
plt.plot([x / 3600.0 for x in t], U_Batt)
  
plt.figure()
plt.xlabel('t [h]')
plt.ylabel('$I_{Batt}$ [mA]')
plt.plot([x / 3600.0 for x in t], [x * 1000.0 for x in I_Batt])  
    
plt.figure()
plt.xlabel('$E_{Batt}$ [Wh]')
plt.ylabel('$U_{Batt}$ [V]')
plt.plot([x / 3600.0 for x in E_Batt], U_Batt)

## Example 4: Adaptive Intermittent Resistor Load
1. Discharge an Alkaline battery with an intermittent $6\Omega$ resistor load ($T_{ON}$ = $T_{OFF}$ = 50s)
2. Discharge an Alkaline battery with an adaptive intermittent $6\Omega$ resistor load. Start with $T_{ON} = T_{OFF} = 50s$ until $U_{Batt}$ falls bellow $U_{thresh} = 1V$. Then, continue with $T_{ON} = T_{OFF} = 1s$.

In [ ]:
plt.figure()
plt.xlabel('t [h]')
plt.ylabel('$U_{Batt}$ [V]')

# not adaptive
#  -> always: T_ON = T_OFF = 50s
load = ResistorLoad(6.0, 50.0, 50.0)
t, E_Batt, U_Batt, I_Batt = ISimulator(Alkaline(), load).run()
plt.plot([x / 3600.0 for x in t], U_Batt, 'r-')

# adaptive: U_thresh = 1.0 V
#  -> start: T_ON = T_OFF = 50s
#  -> save:  T_ON = T_OFF = 1s
load = AdaptiveResistorLoad(1.0,\
                            6.0, 50.0, 50.0,\
                            6.0, 1.0,  1.0)
t, E_Batt, U_Batt, I_Batt = ISimulator(Alkaline(), load).run()
plt.plot([x / 3600.0 for x in t], U_Batt, 'b-')

## Example 5: Compare Simulation with Measurement
Compare a simulation result to measurement data and print error stats. 

NOTE: Make sure to use the same `step` for simulation and measurement data.

NOTE: Unzip `/measurements/*.zip` and run `/measurements/filter.py` first to prepare the measurement data.

In [ ]:
# experiment parameters
step = 0.1
R = 24.8
T_ON = 50.0
T_OFF = 50.0

# simulate
t, E_Batt, U_Batt, _ = ISimulator(LiPo(), ResistorLoad(R, T_ON, T_OFF)).run(step, mstep = 10.0)
  
# load measurement
t_real, U_Batt_real = CSV.load2("../measurements/24.8ohm_50s_50s_lipo-savgol7.csv", step)
E_Batt_real = get_energy(t_real, U_Batt_real, R, T_ON, T_OFF)
  
# plot curves
plt.figure()
plt.xlabel('t [h]')
plt.ylabel('$U_{Batt}$ [V]')
plt.plot([x / 3600.0 for x in t], U_Batt, 'b-')
plt.plot([x / 3600.0 for x in t_real], U_Batt_real, 'r-')

plt.figure()  
plt.xlabel('$E_{Batt}$ [Wh]')
plt.ylabel('$U_{Batt}$ [V]')
plt.plot([x / 3600.0 for x in E_Batt], U_Batt, 'b-')
plt.plot([x / 3600.0 for x in E_Batt_real], U_Batt_real, 'r-')
  
# print error stats
stats("24.8ohm_50s_50s_lipo", t_real, t, U_Batt_real, U_Batt, E_Batt_real, E_Batt)  